# 鏡頭校正

# 圖片偵測

In [21]:
import cv2
from ultralytics import YOLO
import numpy as np
import importlib
import cube_detector as CD
import pickle

model = YOLO("yolov8n-seg-custom.pt")



In [22]:
def draw_axis(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(np.intp(imgpts[0].ravel())), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(np.intp(imgpts[1].ravel())), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(np.intp(imgpts[2].ravel())), (0,0,255), 5)
    return img

In [23]:
importlib.reload(CD)

detector=CD.cubeDetector(model,isCudaSupport=True,processing_hegiht=300)

In [20]:
# img = cv2.imread("test1.jpg")
img = cv2.imread("pnp_test_img/img5.png")
detector.isCudaSupport=True
height =480
scale = height / img.shape[0]
img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
img2  = detector.detect(img,index=None,color=None,show_process_img=False)


cube_imagePoints=detector.get_cube_largest_surface_imagePoints("green")
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cube_imagePoints=cv2.cornerSubPix(gray_image,cube_imagePoints.astype(np.float32),(5,5),(-1,-1),criteria)
cube_objectPoints=np.array([(0,0,0),(0,25,0),(25,25,0),(25,0,0)])
with open('camera/calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)
with open('table_points.pkl','rb') as file:
    table_objectPoints,table_imagePoints = pickle.load(file)
with open('table_solvePnP.pkl','rb') as file:
    table_rvec,table_tvec = pickle.load(file)




    

success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints.astype(float),cube_imagePoints,mtx,dist)
# ret,table_rvecs,table_tvecs = cv2.solvePnP(table_objectPoints,table_imagePoints,mtx,dist)

axis = np.float32([[25,0,0], [0,25,0], [0,0,25]]).reshape(-1,3)
axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
img2=draw_axis(img2,cube_imagePoints.astype(int),axis_cube_img_points)
img2=draw_axis(img2,table_imagePoints.astype(int),axis_table_img_points)
if success:
    # 使用rvec和tvec将物体坐标从世界坐标系转换到相机坐标系
    rotation_matrix, _ = cv2.Rodrigues(cube_rvec)
    transform_matrix = np.column_stack((rotation_matrix, cube_tvec))
    transform_matrix = np.row_stack((transform_matrix,np.array([0,0,0,1])))
    # print(transform_matrix)
    # transformed_point = np.dot(transform_matrix, np.array([12.5, 12.5, -12.5, 1]))
    transformed_point = np.dot(transform_matrix, np.array([12.5, 12.5, -12.5, 1]))
    rotation_matrix, _ = cv2.Rodrigues(table_rvec)
    rotation_matrix=np.linalg.inv(rotation_matrix)
    transformed_point = np.dot(rotation_matrix,(transformed_point[:-1]-table_tvec.T).T)
    

    # 输出结果
    print(cube_tvec)
    print("物体在相机坐标系中的坐标:", transformed_point)

cv2.imshow("test", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 480x640 2 cubes, 80.0ms
Speed: 3.0ms preprocess, 80.0ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)


[     20.392      12.431]
[     10.892      285.97]
[     209.38      297.71]
[      217.7      10.289]
[         13         210]
[     246.88      281.77]
[        275         116]
[     232.88      17.231]
[         81         142]
[     45.786      291.95]
[     27.204      22.207]
[[     100.08]
 [    -26.558]
 [     352.83]]
物体在相机坐标系中的坐标: [[     173.81]
 [     127.43]
 [     5.5596]]


# 測試

In [327]:
import cv2

cap = cv2.VideoCapture(0)

num = 1

while cap.isOpened():

    succes, img = cap.read()

    k = cv2.waitKey(5)

    if k == ord('q'):
        break
    elif k == ord('s'): # wait for 's' key to save and exit
        cv2.imwrite('pnp_test_img/img' + str(num) + '.png', img)
        print("image saved!")
        num += 1

    cv2.imshow('Img',img)

# Release and destroy all windows before termination
cap.release()

cv2.destroyAllWindows()

image saved!
image saved!
image saved!
image saved!
image saved!
image saved!


# 找桌面座標

## V1

In [316]:
import cv2

# 创建一个空白图像

image = cv2.imread('pnp_test_img/img0.png')  # 替换为您的图像文件路径

drawing_image= image.copy()
Adjusted_image= None

isAdjusting=False
isDrawing = True  # 标记是否正在绘制线段
isFirstDraw=True
square_points = []
line_thickness=2
# 回调函数，用于处理鼠标事件
def draw_line(event, x, y, flags, param):
    global  isDrawing,drawing_image,Adjusted_image,square_points,isAdjusting,isFirstDraw,line_thickness
    if event == cv2.EVENT_LBUTTONDOWN:  # 鼠标左键按下事件
        if isDrawing:
            if isFirstDraw:
                isFirstDraw = False
            else :
                cv2.line(drawing_image,square_points[-1],(x,y),(255,0,0),2)
            if len(square_points) < 4:
                square_points.append((x,y))
            if len(square_points)==4:
                isDrawing = False
                isAdjusting = True
                Adjusted_image=image.copy()
                cv2.polylines(Adjusted_image, [np.array(square_points)], isClosed=True, color=(0, 255, 0), thickness=2)
                
        elif isAdjusting:
            distances = np.linalg.norm(np.array(square_points) - np.array((x,y)), axis=1)
            nearest_index = np.argmin(distances)
            square_points[nearest_index]=(x,y)
            Adjusted_image=image.copy()
            cv2.polylines(Adjusted_image, [np.array(square_points)], isClosed=True, color=(0, 255, 0), thickness=2)
    elif event == cv2.EVENT_MOUSEMOVE:  # 鼠标移动事件
        if isDrawing and not isFirstDraw:
            temp_image = drawing_image.copy()  # 复制图像以防止绘制覆盖
            cv2.line(temp_image, square_points[-1], (x, y), (0, 255, 0), line_thickness)
            cv2.imshow('Image with Lines', temp_image)
        if isAdjusting:
            cv2.imshow('Image with Lines', Adjusted_image)


# 创建一个窗口并将鼠标事件处理函数绑定到它
cv2.namedWindow('Image with Lines')
cv2.setMouseCallback('Image with Lines', draw_line)
# 在窗口中显示图像，并等待用户按下ESC键退出
while True:
    if not isAdjusting:
        cv2.imshow('Image with Lines', drawing_image)
    else:
        cv2.imshow('Image with Lines', Adjusted_image)
    key = cv2.waitKey(1)
    if key == 27:  # 如果按下ESC键，退出循环
        break
   
cv2.destroyAllWindows()


with open('table_points.pkl', 'wb') as file:
        pickle.dump((np.float32([(0,0),(custom_width,0),(custom_height,custom_width),(0,custom_height)]),
                     np.float32(square_points)), file)

# with open('table_points.pkl', 'rb') as file:
#     img_points,object_points = pickle.load(file)
#     print(img_points)
#     print(object_points)

# V2

In [29]:
#超屌抓桌面程式


import cv2

# 创建一个空白图像
square_height,square_width = setHeightWin()
square_coordinates=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
with open('camera/calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)

image = cv2.imread('pnp_test_img/img2.png')  # 替换为您的图像文件路径

drawing_image= image.copy()
Adjusted_image= None

isAdjusting=False
isDrawing = True  # 标记是否正在绘制线段
isFirstDraw=True
square_points = []
line_thickness=2
# 回调函数，用于处理鼠标事件
def draw_line(event, x, y, flags, param):
    global line_start, line_end, isDrawing,drawing_image,Adjusted_image,square_points,isAdjusting,isFirstDraw,line_thickness
    if event == cv2.EVENT_LBUTTONDOWN:  # 鼠标左键按下事件
        if isDrawing:
            if isFirstDraw:
                isFirstDraw = False
            else :
                cv2.line(drawing_image,square_points[-1],(x,y),(255,0,0),line_thickness)
            if len(square_points) < 4:
                square_points.append((x,y))
            if len(square_points)==4:
                isDrawing = False
                isAdjusting = True
                frame_update()
        elif isAdjusting:
            distances = np.linalg.norm(np.array(square_points) - np.array((x,y)), axis=1)
            nearest_index = np.argmin(distances)
            square_points[nearest_index]=(x,y)
            frame_update()
    elif event == cv2.EVENT_MOUSEMOVE:  # 鼠标移动事件
        if isDrawing and not isFirstDraw:
            temp_image = drawing_image.copy()  # 复制图像以防止绘制覆盖
            cv2.line(temp_image, square_points[-1], (x, y), (0, 255, 0), line_thickness)
            cv2.imshow('Image with Lines', temp_image)
        if isAdjusting:
            cv2.imshow('Image with Lines', Adjusted_image)

    elif event == cv2.EVENT_MOUSEWHEEL:
        delta = flags
        if delta >0:
            if line_thickness==3:
                pass
            else:
                line_thickness+=1
        elif delta<0:
            if line_thickness == 1:
                pass
            else:
                line_thickness-=1
        frame_update()

# 创建一个窗口并将鼠标事件处理函数绑定到它
cv2.namedWindow('Image with Lines')
cv2.setMouseCallback('Image with Lines', draw_line)
def frame_update():
    global Adjusted_image, square_points,line_thickness,rvecs,tvecs
    Adjusted_image=image.copy()
    cv2.polylines(Adjusted_image, [np.array(square_points)], isClosed=True, color=(0, 255, 0), thickness=line_thickness)
    for coordinate,point in zip(square_coordinates,square_points):
        cv2.putText(Adjusted_image, f"{list(coordinate)}", point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 255), 2)
    square_img_points = np.float32(square_points)
    # print(square_img_points)
    ret,rvecs,tvecs = cv2.solvePnP(square_coordinates,square_img_points,mtx,dist)
    # print(rvecs)
    axis = np.float32([[100,0,0], [0,100,0], [0,0,100]]).reshape(-1,3)
    axis_img_points, _ = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    draw_axis(Adjusted_image,square_img_points.astype(int),axis_img_points)
# 在窗口中显示图像，并等待用户按下ESC键退出
while True:
    if not isAdjusting:
        cv2.imshow('Image with Lines', drawing_image)
    else:
        cv2.imshow('Image with Lines', Adjusted_image)
    key = cv2.waitKey(1)
    if key == 27:  # 如果按下ESC键，退出循环
        break
    if key == ord("r"):
        if len(square_points)==4:
            square_points[1],square_points[3]=square_points[3],square_points[1]
            # square_coordinates[1],square_coordinates[3]=square_coordinates[3],square_coordinates[1]
            frame_update()
    if key == ord("e"):
        if len(square_points)==4:
            square_points=square_points[-1:]+ square_points[:-1] #前包後不包
            frame_update()
    if key == ord("w"):
        square_height,square_width = setHeightWin()
        square_coordinates=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
        frame_update()
    if key ==ord("q"):
        square_height,square_width=square_width,square_height
        square_coordinates=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
        frame_update()
    if key == ord("s"):
        with open('table_points.pkl', 'wb') as file:
                pickle.dump((square_coordinates,np.float32(square_points)), file)
        with open('table_solvePnP.pkl', 'wb') as file:
                pickle.dump((rvecs,tvecs), file)
        print("saved")
                    
cv2.destroyAllWindows()



# with open('table_points.pkl', 'rb') as file:
#     img_points,object_points = pickle.load(file)
#     print(img_points)
#     print(object_points)

In [24]:
import tkinter as tk

# 创建主窗口
def setHeightWin():
    root = tk.Tk()
    root.title("Input Form")

    # 获取屏幕的宽度和高度
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # 设置窗口的宽度和高度
    window_width = 400  # 例如，设置窗口宽度为400像素
    window_height = 150  # 例如，设置窗口高度为300像素

    # 计算窗口的左上角坐标，使其显示在屏幕正中央
    x = (screen_width - window_width) // 2
    y = (screen_height - window_height) // 2

    # 设置窗口的位置
    root.geometry(f"{window_width}x{window_height}+{x}+{y}")

    # 创建Label和Entry组件来获取用户输入
    label_height = tk.Label(root, text="Height:", font=("Helvetica", 14))  # 设置字体大小为14
    label_height.pack()

    entry_height = tk.Entry(root, font=("Helvetica", 14),justify="center")  # 设置字体大小为14
    entry_height.pack()

    label_width = tk.Label(root, text="Width:", font=("Helvetica", 14))  # 设置字体大小为14
    label_width.pack()

    entry_width = tk.Entry(root, font=("Helvetica", 14),justify="center")  # 设置字体大小为14
    entry_width.pack()
    
    result = []
    # 创建一个函数，用于处理用户点击"送出"按钮时的操作
    def submit_form():
        height = entry_height.get()
        width = entry_width.get()
        

        result.extend([height, width])

        # 在这里，您可以处理获取的height和width数据，例如打印它们
        root.destroy()

    # 创建"送出"按钮
    root.bind('<Return>', lambda event=None: submit_form())
    submit_button = tk.Button(root, text="送出", command=submit_form,width=10,height=1,font=("Helveticsa",14))
    submit_button.pack()

    # 启动Tkinter主循环
    root.mainloop()
    return result